In [530]:
import pandas as pd
out=pd.read_table('csdn_blog_100.csv',sep='\t',
                engine='python',index_col=['_id'],header=None,names=['_id','标题','标签','时间','pv','作者','原创','c'])
out['nlp']=''

In [549]:
import jieba
import jieba.analyse
import jieba.posseg as pseg
import codecs
from bs4 import BeautifulSoup
from gensim import corpora, models, similarities

# 加停用词库
stopwords = codecs.open('../words/CNstop_words_zh.txt','r',encoding='utf8').readlines()
stopwordlist = [ w.strip() for w in stopwords ]
# 结巴分词后的停用词性 [标点符号、连词、助词、副词、介词、时语素、‘的’、数词、方位词、代词]
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

In [532]:
def tokenization(doc):
#     结巴分词，去掉停用词
    result = []
    text = doc
    words = pseg.cut(text)
    for word, flag in words: 
        if flag not in stop_flag and word not in stopwords: 
            result.append(word)
    return result

ids=[]
dockeys=[]
# 100篇文档分词，保存
for row in out.itertuples():
    soup = BeautifulSoup(row.c,'html5lib').getText().strip()
    ids.append(row.Index)
    word_list = tokenization(soup)
    dockeys.append(word_list)
    out.loc[out.index==row[0],['nlp']] = ' '.join([w for w in word_list])

out.drop(out.columns[[6]], axis=1,inplace=True)
out.to_csv('csdn_blog_100_nlp.csv',sep='\t')
try:
#     保存文档id列表
    import pickle
    f = open("./csdn_blog_ids_100.dat", "wb")
    pickle.dump(ids, f)
    f.close()
except Exception as e:
    print('csdn_blog_ids_100.dat保存文件出错')
    
try:
#     保存文档dockeys列表
    import pickle
    f = open("./csdn_blog_dockeys_100.dat", "wb")
    pickle.dump(dockeys, f)
    f.close()
except Exception as e:
    print('csdn_blog_dockeys_100.dat保存文件出错')

In [533]:
from gensim import corpora, models, similarities
# 建立词袋模型
dictionary = corpora.Dictionary(dockeys)
dictionary.save('csdn_blog_100.dict')
# print(dictionary)
# print(len(dockeys))
# for doc in dockey:
#     print(len(doc))
# 生成词向量
corpus = [dictionary.doc2bow(text) for text in dockeys]
corpora.MmCorpus.serialize('csdn_blog_100.mm', corpus)
# print('len',len(corpus))
# print(doc_vectors)
# for doc in corpus:
#     print(len(doc))

In [550]:
# 建立TF-IDF模型
tfidf = models.TfidfModel(corpus)
tfidf_corpus = tfidf[corpus]
# print(len(tfidf_corpus))
# print(len(tfidf_vectors[0]))
# for doc in tfidf_vectors:
#     print(doc)
# 持久化
tfidf.save("./csdn_blog_model_100.tfidf")
tfidf = models.TfidfModel.load("./csdn_blog_model_100.tfidf")

In [551]:
# 生成tf/idf模型,相似度矩阵
index_tfidf = similarities.MatrixSimilarity(tfidf_corpus)
index_tfidf.save('csdn_blog_100.index')

def tfidfsimilarities(f,index,N=20):
    # 求tf/idf模型,相似度
    doc=''
    with open(f, 'r') as f:
        doc = f.read().replace('\n','')
    query=tokenization(doc)
    query_bow=dictionary.doc2bow(query)
    sims = index[query_bow]
    from operator import itemgetter, attrgetter
    return sorted(list(enumerate(sims)), key=itemgetter(1),reverse=True)[0:N]

def toids(sim,ids):
#     语料库ID转换成数据库里的_id
    return [(ids[s[0]],s[1]) for s in sim]

sim1 = tfidfsimilarities('../测试样本/1.txt',index_tfidf)
toids(sim1,ids)

[(4092, 0.17394318),
 (10107, 0.13801867),
 (6112, 0.12311725),
 (4461, 0.11609715),
 (321, 0.10852387),
 (1883, 0.094831996),
 (4926, 0.093249574),
 (4137, 0.079429373),
 (12345, 0.076761037),
 (5670, 0.076752216),
 (10364, 0.07548704),
 (823, 0.070422918),
 (12111, 0.069211349),
 (2606, 0.064810753),
 (12242, 0.063527085),
 (10970, 0.062913775),
 (1823, 0.061469492),
 (4485, 0.060431741),
 (322, 0.057909347),
 (1834, 0.05714741)]

In [579]:
# 构建LSI模型，设置主题数为10
lsi = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=10)
lsi.print_topics(4)
lsi_corpus = lsi[tfidf_corpus]
# 生成lsi模型,相似度矩阵
index_lsi = similarities.MatrixSimilarity(lsi_corpus)
index_lsi.save('csdn_blog_100_lsi.index')

def sim(f,Index,L,N=20):
    # 求tf/idf模型,相似度
    doc=''
    with open(f, 'r') as f:
        doc = f.read().replace('\n','')
    query=tokenization(doc)
    query_bow=dictionary.doc2bow(query)
    query = L[query_bow]
    sims = Index[query]
    from operator import itemgetter, attrgetter
    return sorted(list(enumerate(sims)), key=itemgetter(1),reverse=True)[0:N]

In [576]:
sim2 = sim('../测试样本/2.txt',index_lsi,lsi)
# sim3 = similarities('../测试样本/3.txt',index_lsi)
# sim4 = similarities('../测试样本/4.txt',index_lsi)
# sim5 = similarities('../测试样本/5.txt',index_lsi)
toids(sim2,ids)

[(10982, 0.85179329),
 (1996, 0.84430254),
 (7617, 0.83874285),
 (9975, 0.80283195),
 (2872, 0.80068243),
 (3825, 0.79446274),
 (10440, 0.78203481),
 (3291, 0.78144395),
 (1999, 0.77139902),
 (10708, 0.76999611),
 (12093, 0.76862645),
 (10128, 0.76752114),
 (3000, 0.75083762),
 (4765, 0.74972397),
 (4092, 0.73884326),
 (5621, 0.69803166),
 (4137, 0.6779803),
 (7795, 0.66565019),
 (3293, 0.65692222),
 (3290, 0.65393209)]

In [609]:
# 构建LDA模型，设置主题数为10
lda = models.LdaModel(tfidf_corpus, id2word=dictionary, num_topics=5)
# lsi.print_topics(4)
lda_corpus = lda[tfidf_corpus]
# 生成lda模型,相似度矩阵
index_lda = similarities.MatrixSimilarity(lda_corpus)
index_lda.save('csdn_blog_100_lda.index')

In [610]:
sim2 = sim('../测试样本/2.txt',index_lda,lda)
toids(sim2,ids)

[(12242, 0.96680576),
 (4653, 0.94866389),
 (3291, 0.94818997),
 (10084, 0.94552886),
 (9401, 0.94524658),
 (11918, 0.94314307),
 (3293, 0.94303781),
 (9446, 0.94299531),
 (11771, 0.94168556),
 (2600, 0.94094592),
 (3998, 0.94028157),
 (10982, 0.93924826),
 (7795, 0.93915576),
 (9975, 0.93859255),
 (638, 0.93847704),
 (5402, 0.93817246),
 (6112, 0.93769437),
 (2604, 0.93706495),
 (2606, 0.93669379),
 (10970, 0.9366678)]

In [606]:
# 构建HDP模型
hdp = models.HdpModel(tfidf_corpus, id2word=dictionary)
# lsi.print_topics(4)
hdp_corpus = hdp[tfidf_corpus]
# 生成hdp模型,相似度矩阵
index_hdp = similarities.MatrixSimilarity(hdp_corpus, num_features=hdp.m_T)
index_hdp.save('csdn_blog_100_hdp.index')

In [607]:
sim2 = sim('../测试样本/2.txt',index_hdp,hdp)
toids(sim2,ids)

[(10314, 0.1671586),
 (11083, 0.1671586),
 (321, 0.0),
 (322, 0.0),
 (638, 0.0),
 (823, 0.0),
 (1690, 0.0),
 (1796, 0.0),
 (1823, 0.0),
 (1826, 0.0),
 (1834, 0.0),
 (1883, 0.0),
 (1996, 0.0),
 (1999, 0.0),
 (2010, 0.0),
 (2014, 0.0),
 (2165, 0.0),
 (2430, 0.0),
 (2600, 0.0),
 (2601, 0.0)]

In [618]:
sim3 = sim('../测试样本/5.txt',index_hdp,hdp)
toids(sim3,ids)

[(4092, 0.21201377),
 (4873, 0.21201377),
 (3087, 0.179215),
 (10614, 0.17920174),
 (4096, 0.15615825),
 (3541, 0.15540251),
 (3998, 0.14279126),
 (3338, 0.14110023),
 (4288, 0.13935466),
 (6960, 0.13181487),
 (2165, 0.12796322),
 (1823, 0.12772173),
 (4335, 0.12734564),
 (8543, 0.11255755),
 (7617, 0.10724392),
 (2606, 0.093235143),
 (9904, 0.088577986),
 (10466, 0.085759655),
 (10084, 0.081613101),
 (3796, 0.079113439)]

In [602]:
index_hdp

In [603]:
for i in index_hdp:
    print(i)

[ 1.          0.25042129  0.17548548  0.3157534   0.          0.          0.
  0.          0.46536458  0.23692591  0.22600213  0.23370574  0.          0.
  0.4101252   0.          0.          0.37101144  0.          0.
  0.40429533  0.32936156  0.66732883  0.          0.          0.41595834
  0.          0.          0.          0.53516012  0.38507584  0.          0.
  0.32266441  0.29433498  0.          0.2348541   0.          0.21582232
  0.46404773  0.46074799  0.43042082  0.22281262  0.          0.
  0.00674771  0.40009949  0.25425535  0.          0.          0.          0.
  0.35842648  0.36704838  0.41393816  0.43128031  0.36387736  0.4002617   0.
  0.          0.64756173  0.32468292  0.          0.23924056  0.
  0.29831401  0.          0.30614498  0.          0.          0.37741002
  0.          0.          0.          0.32961953  0.21140745  0.22707471
  0.53512001  0.40207165  0.          0.30487427  0.53516012  0.21608031
  0.          0.          0.          0.          0.535

[ 0.          0.          0.          0.7496888   0.43876272  0.2386142
  0.44431958  0.          0.37725776  0.28503147  0.68583739  0.50279099
  0.55890131  0.55890131  0.35904467  0.          0.          0.59760934
  0.98283142  0.          0.36618531  0.4405148   0.          0.31676769
  0.          0.52173764  0.55890131  0.          0.          0.
  0.16077951  0.24456617  0.97916639  0.33346608  0.28350595  0.39319703
  0.50220704  0.17506568  0.33780167  0.27838889  0.30977929  0.4053573   0.
  0.20507684  0.86460865  0.56926823  0.37117767  0.43296519  0.12266349
  0.          0.28287277  0.55890131  0.6157741   0.6929667   0.
  0.33090591  0.          0.          0.          0.55890131  0.          0.
  0.          0.44919521  0.          0.          0.          0.36231339
  0.          0.          0.19272833  0.99505627  0.00692772  1.
  0.32005247  0.41779259  0.46123552  0.00684429  0.          0.39507958
  0.45933923  0.          0.69930679  0.          0.55890131  0.
  0